# Startegi : Using each label using neural network(regression) and choose predictor by correlation
## tunning parameter

In [5]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [6]:
path='data/train.csv'
list_variable={'x':['sat_id', 'x_sim', 'y_sim', 'Vz_sim'], 'y':['sat_id','x_sim','y_sim','Vx_sim'], 'z':['sat_id','y_sim','z_sim','Vy_sim']}

In [8]:
df=pd.read_csv(path, index_col=0)
# df=df[['x','y','z','x_sim','y_sim','z_sim']]
df=df.drop('epoch', axis=1)
df.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,
0,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [9]:
mydata=df.copy()

In [10]:
mydata.isna().sum()

sat_id    0
x         0
y         0
z         0
Vx        0
Vy        0
Vz        0
x_sim     0
y_sim     0
z_sim     0
Vx_sim    0
Vy_sim    0
Vz_sim    0
dtype: int64

In [11]:
mydata = mydata.dropna()
mydata.shape

(649912, 13)

In [12]:
train_mydata = mydata.sample(frac=0.8,random_state=0)
test_mydata = mydata.drop(train_mydata.index)
test_mydata.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,
1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
4,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
15,0,17281.246143,-2011.121378,39970.739873,-0.104355,2.653343,-0.295245,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835
22,0,-1447.724638,28032.228987,-3918.192838,-1.560091,-1.321435,-3.577772,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769
26,0,-10583.521489,-10145.003880,-24243.775722,0.275212,-4.049127,0.719273,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858


In [13]:
#a=train_mydata.shape
#b=test_mydata.shape
#print(a,b)
print(train_mydata.shape, test_mydata.shape)
train_mydata.head()

(519930, 13) (129982, 13)


,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,
162906,81,-22445.306373,5848.139687,805.906588,-1.295831,-3.424371,0.920461,-21786.206588,7375.036638,391.178832,-1.616993,-3.329732,0.928155
123662,60,-9086.781838,7654.307240,-6721.299832,4.189223,1.357308,-2.946352,-9545.479053,7482.025507,-6387.708079,4.036478,1.482306,-3.057046
901720,449,-33475.100342,-37591.455624,-3269.507831,0.115463,-1.791145,-1.901966,-33263.131053,-37783.198553,-3617.165346,0.130353,-1.782338,-1.906924
308669,166,11617.034853,2343.826977,-1821.565305,-1.136521,5.835368,1.405297,10718.558812,5174.769686,-1069.308627,-2.446951,5.391866,1.577077
1154759,560,25113.931453,-473.679010,41867.498593,-0.849950,2.496597,-1.506792,19229.123354,12147.342145,31835.248429,-1.395221,2.336560,-2.422340


In [14]:
label=['Vx','Vy','Vz','x','y','z'][3:]
train_stats = train_mydata.describe()
for i in label:
  train_stats.pop(i)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
sat_id,519930.0,308.718101,171.427627,0.000000,166.000000,312.000000,461.000000,599.000000
Vx,519930.0,-0.001039,2.638712,-8.286673,-1.894451,0.017755,1.958741,8.149559
Vy,519930.0,-0.001091,2.596198,-8.231654,-1.876317,-0.053612,1.840252,12.308450
Vz,519930.0,-0.000599,2.116234,-9.371164,-1.256092,0.008927,1.233894,8.644558
x_sim,519930.0,-941.331006,25387.037253,-255037.952966,-13139.771199,-612.877803,11194.372299,200961.158000
y_sim,519930.0,319.853163,25603.245579,-281784.430040,-11758.892200,501.609913,12558.620911,209619.553857
z_sim,519930.0,-77.075471,20670.833347,-180250.922287,-7963.461284,57.093264,8253.028193,198526.091552
Vx_sim,519930.0,-0.001115,2.636436,-8.240694,-1.896241,0.019026,1.954861,7.431844
Vy_sim,519930.0,-0.002132,2.599643,-8.348831,-1.878077,-0.018243,1.857399,8.021438
Vz_sim,519930.0,-0.000170,2.114104,-7.671565,-1.247046,0.011688,1.235497,8.644165


In [15]:
train_labels = train_mydata[label]
test_labels = test_mydata[label]

In [16]:
print(train_labels.head(), test_labels.head())

                    x             y             z
id                                               
162906  -22445.306373   5848.139687    805.906588
123662   -9086.781838   7654.307240  -6721.299832
901720  -33475.100342 -37591.455624  -3269.507831
308669   11617.034853   2343.826977  -1821.565305
1154759  25113.931453   -473.679010  41867.498593                x             y             z
id                                          
1  -10567.672384   1619.746066 -24451.813271
4   -6719.092336 -28929.061629 -14938.907967
15  17281.246143  -2011.121378  39970.739873
22  -1447.724638  28032.228987  -3918.192838
26 -10583.521489 -10145.003880 -24243.775722


In [17]:
def norm(x):
  try:
    return (x - train_stats['mean']) / train_stats['std']
  except:
    print(x)

In [18]:
normed_train_data = norm(train_mydata.drop(label, axis=1))
normed_test_data = norm(test_mydata.drop(label, axis=1))

In [19]:
a=normed_test_data.describe()
a.transpose()

,count,mean,std,min,25%,50%,75%,max
sat_id,129982.0,0.000357,1.001616,-1.800866,-0.832527,0.019145,0.888316,1.693320
Vx,129982.0,0.000887,1.000627,-3.136737,-0.713950,0.010402,0.745366,3.085047
Vy,129982.0,0.003002,1.001795,-2.964310,-0.719889,-0.016264,0.719243,3.227379
Vz,129982.0,0.001217,1.000030,-4.002201,-0.592784,0.005625,0.590960,4.077451
x_sim,129982.0,-0.001650,0.997447,-10.002493,-0.483600,0.011332,0.477304,7.940276
y_sim,129982.0,0.000919,1.000829,-11.059974,-0.467484,0.011119,0.476295,8.154777
z_sim,129982.0,-0.000651,0.999315,-8.628536,-0.380103,0.006436,0.399140,9.581585
Vx_sim,129982.0,0.001130,1.002003,-3.126207,-0.718826,0.011815,0.743078,2.819324
Vy_sim,129982.0,0.004858,1.001302,-3.209618,-0.719916,0.000971,0.722968,3.079462
Vz_sim,129982.0,-0.000335,0.999210,-3.624755,-0.589240,0.006347,0.585358,4.088883


In [20]:
print(normed_test_data.shape)
normed_test_data.head()

(129982, 10)


,sat_id,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,
1,-1.800866,-0.114280,-1.645301,-0.289286,-0.378704,0.051925,-1.180214,-0.114772,-1.641642,-0.291518
4,-1.800866,0.376527,-0.970127,1.108244,-0.227991,-1.141345,-0.721564,0.375696,-0.969551,1.107991
15,-1.800866,-0.039154,1.022432,-0.139231,0.716221,-0.091263,1.936827,-0.038963,1.022244,-0.139380
22,-1.800866,-0.590838,-0.508568,-1.690348,-0.020474,1.082138,-0.187834,-0.590174,-0.508629,-1.692253
26,-1.800866,0.104692,-1.559217,0.340167,-0.379433,-0.408845,-1.170397,0.104497,-1.555827,0.340110


In [26]:
def create_model(input_dim):
    NN_model = Sequential()

    # The Input Layer :
    NN_model.add(Dense(100, kernel_initializer='normal',input_dim = input_dim, activation='relu'))

    # The Hidden Layers :
    NN_model.add(Dense(200, kernel_initializer='normal',activation='relu'))
    NN_model.add(Dense(200, kernel_initializer='normal',activation='relu'))
    NN_model.add(Dense(200, kernel_initializer='normal',activation='relu'))

    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    # Compile the network :
    NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    NN_model.summary()
    return NN_model

In [27]:
list_variable

{'x': ['sat_id', 'x_sim', 'y_sim', 'Vz_sim'],
 'y': ['sat_id', 'x_sim', 'y_sim', 'Vx_sim'],
 'z': ['sat_id', 'y_sim', 'z_sim', 'Vy_sim']}

In [28]:
train_labels.head()

,x,y,z
id,,,
162906,-22445.306373,5848.139687,805.906588
123662,-9086.781838,7654.307240,-6721.299832
901720,-33475.100342,-37591.455624,-3269.507831
308669,11617.034853,2343.826977,-1821.565305
1154759,25113.931453,-473.679010,41867.498593


In [29]:
test_mydata.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,
1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
4,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
15,0,17281.246143,-2011.121378,39970.739873,-0.104355,2.653343,-0.295245,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835
22,0,-1447.724638,28032.228987,-3918.192838,-1.560091,-1.321435,-3.577772,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769
26,0,-10583.521489,-10145.003880,-24243.775722,0.275212,-4.049127,0.719273,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858


In [30]:
for i in list_variable:
    train=normed_train_data[list_variable[i]]
    label_train=train_labels[i].values
    print(i, train.columns)
    test=normed_test_data[list_variable[i]]
    label_test=test_labels[i].values
    print(len(train.columns))
    model=create_model(input_dim=len(train.columns))
    model.fit(train, label_train, validation_data=(test, label_test), epochs=10)
    test_mydata[i+'_pred']=model.predict(test)

x Index(['sat_id', 'x_sim', 'y_sim', 'Vz_sim'], dtype='object')
4
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_7 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_8 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_9 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 201       
Total params: 101,301
Trainable params: 101,301
Non-trainable params: 0
_________________________________________________________________
Train on 519930 samples, validate on 129982 samp

KeyboardInterrupt: 

In [76]:
test_mydata.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_pred,y_pred,z_pred
id,,,,,,,,,,,,,,,,
1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,-9183.037109,1851.521240,-24268.890625
4,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,-8604.287109,-28910.746094,-15109.281250
15,0,17281.246143,-2011.121378,39970.739873,-0.104355,2.653343,-0.295245,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835,16839.861328,-2926.081055,40413.890625
22,0,-1447.724638,28032.228987,-3918.192838,-1.560091,-1.321435,-3.577772,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769,1686.144043,27366.716797,-3611.077881
26,0,-10583.521489,-10145.003880,-24243.775722,0.275212,-4.049127,0.719273,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858,-11038.967773,-9878.979492,-24278.093750


In [77]:
def smape(satellite_predicted_values, satellite_true_values):
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/(np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [79]:
for i in ['_sim', '_pred']:
    real=test_mydata[label].values
    comp=test_mydata[[j+i for j in label]].values
    print('smape real vs %s : %f'%(i, smape(real, comp)))

smape real vs _sim : 0.184052
smape real vs _pred : 0.182019
